Imports and Setup

In [ ]:
"""
# HTB Attack Chain with LangChain
**Author**: Ella Duffy  
**Version**: 1.1.1  
**Description**: Automates the attack chain for HTB challenges using LangChain.
**Note**: This is the legacy chain-based approach. Transitioning to react_agent.ipynb for modern ReAct agent implementation.
"""

import logging
import time
from langchain.chains import SequentialChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain.schema.runnable import RunnablePassthrough, RunnableSequence, RunnableLambda
from langchain.memory import ConversationBufferMemory
from ssh_command_executor import Tools

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('logs/attack_chain.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

logger.info("Loading HTB Attack Chain with LangChain")

# Initialize LLM
llm = OpenAI(model_name="gpt-4o-mini", temperature=0)

#Initialize memory
memory = ConversationBufferMemory()

# Initialize Tools
tools = Tools(memory=memory)

2025-07-04 16:53:43,381 - INFO - Loading SSH Command Executor
2025-07-04 16:53:43,384 - INFO - Loading HTB Attack Chain with LangChain


Define Chains

In [4]:
# Port Scanning Chain
port_scan_prompt = PromptTemplate(
    input_variables=["target"],
    template="Scan ports on target {target} and return a dictionary of open ports and services."
)
#port_scan_chain = LLMChain(
#    llm=llm,
#    prompt=port_scan_prompt,
#    output_key="port_scan_output",
#    verbose=True
#)

#Use LangChain Expression Language instead
port_scan_chain = port_scan_prompt | llm

# User Enumeration Chain
user_enum_prompt = PromptTemplate(
    input_variables=["target", "port_scan_output"],
    template="Enumerate users on target {target} if finger service (port 79) is detected in {port_scan_output}."
)
#user_enum_chain = LLMChain(
#    llm=llm,
#    prompt=user_enum_prompt,
#    output_key="user_enum_output",
#    verbose=True
#)
user_enum_chain = {"target": RunnablePassthrough(), "port_scan_output": lambda x: x["port_scan_output"]} | user_enum_prompt | llm

# SSH Port Detection Chain
ssh_detect_prompt = PromptTemplate(
    input_variables=["target", "port_scan_output"],
    template="Detect the SSH port on target {target} using the port scan data {port_scan_output} or fast nmap scan."
)
ssh_detect_chain = {"target": RunnablePassthrough(), "port_scan_output": lambda x: x["port_scan_output"]} | ssh_detect_prompt | llm


Execution Functions

In [5]:
# Cell 3: Execution Functions
def execute_port_scan(inputs):
    target = inputs["target"]
    host = inputs["host"]
    username = inputs["username"]
    logger.info(f"Executing port scan on {target}")
    ports = tools.scan_ports(host, username, target)
    return {**inputs, "port_scan_output": ports}

def execute_user_enum(inputs):
    target = inputs["target"]
    host = inputs["host"]
    username = inputs["username"]
    port_scan_output = inputs["port_scan_output"]
    logger.info(f"Executing user enumeration on {target}")
    if "79" in port_scan_output:
        users = tools.enumerate_users(host, username, target)
        return {**inputs, "user_enum_output": users}
    else:
        logger.warning("Finger service not detected; skipping user enumeration")
        return {**inputs, "user_enum_output": []}
    
def execute_ssh_detect(inputs):
    target = inputs["target"]
    host = inputs["host"]
    username = inputs["username"]
    logger.info(f"Executing SSH port detection on {target}")
    ssh_port = tools.detect_ssh_port(host, username, target)
    return {**inputs, "ssh_port": ssh_port}


Define the runnable sequence

In [6]:
attack_chain = (
    RunnableLambda(lambda inputs: execute_port_scan(inputs))
    | RunnableLambda(lambda inputs: execute_user_enum(inputs))
    | RunnableLambda(lambda inputs: execute_ssh_detect(inputs))
).with_config(verbose=True)

def run_attack_chain(host, username, target):
    logger.info(f"Starting attack chain on {target} via {host}")
    inputs = {
        "target": target,
        "host": host,
        "username": username
    }
    result = attack_chain.invoke(inputs)
    logger.info(f"Attack chain completed: {result}")
    return result

In [7]:
if __name__ == "__main__":
    # Run the chain
    host = "10.0.0.215"
    username = "user"
    target = "10.10.10.76"
    result = run_attack_chain(host, username, target)
    print("Final result:", result)

2025-07-04 16:53:44,341 - INFO - Starting attack chain on 10.10.10.76 via 10.0.0.215
2025-07-04 16:53:44,368 - INFO - Executing port scan on 10.10.10.76
2025-07-04 16:53:44,370 - INFO - Scanning open ports on 10.10.10.76
2025-07-04 16:53:44,373 - INFO - Connecting to 10.0.0.215 as user
2025-07-04 16:53:44,413 - INFO - Connected (version 2.0, client OpenSSH_9.2p1)
2025-07-04 16:53:44,739 - INFO - Authentication (publickey) successful!
2025-07-04 16:53:44,740 - INFO - Executing command: PATH=$PATH:/sbin:/usr/sbin nmap -p1-1000,22022 10.10.10.76 --max-retries 2 -Pn --max-rate 50 --open
2025-07-04 16:54:08,207 - INFO - Command output:
 Starting Nmap 7.94SVN ( https://nmap.org ) at 2025-07-04 20:52 UTC
Nmap scan report for 10.10.10.76
Host is up (0.065s latency).
Not shown: 887 closed tcp ports (conn-refused), 110 filtered tcp ports (no-response)
Some closed ports may be reported as filtered due to --defeat-rst-ratelimit
PORT      STATE SERVICE
79/tcp    open  finger
111/tcp   open  rpcbind

Final result: {'target': '10.10.10.76', 'host': '10.0.0.215', 'username': 'user', 'port_scan_output': {'79': 'finger', '111': 'rpcbind', '515': 'printer', '22022': 'unknown'}, 'user_enum_output': ['sammy', 'sunny'], 'ssh_port': 22022}
